### Neural networks with tensorflow:

The data pre-processing is based on this notebook: https://www.kaggle.com/hmendonca/starter-eda-and-feature-selection-ashrae3

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os, gc
import random
import datetime

from tqdm import tqdm_notebook as tqdm

# matplotlib and seaborn for plotting
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn import preprocessing

import lightgbm as lgb

In [ ]:
path = '../input/ashrae-energy-prediction'
# Input data files are available in the "../input/" directory.
for dirname, _, filenames in os.walk(path):
    for filename in filenames:
        print(os.path.join(dirname, filename))

### Function to load the data:

In [ ]:
%%time
# unimportant features (see importance below)
unimportant_cols = ['wind_direction', 'wind_speed', 'sea_level_pressure']
target = 'meter_reading'

def load_data(source='train', path=path):
    ''' load and merge all tables '''
    assert source in ['train', 'test']
    
    building = pd.read_csv(f'{path}/building_metadata.csv', dtype={'building_id':np.uint16, 'site_id':np.uint8})
    weather  = pd.read_csv(f'{path}/weather_{source}.csv', parse_dates=['timestamp'],
                                                           dtype={'site_id':np.uint8, 'air_temperature':np.float16,
                                                                  'cloud_coverage':np.float16, 'dew_temperature':np.float16,
                                                                  'precip_depth_1_hr':np.float16},
                                                           usecols=lambda c: c not in unimportant_cols)
    df = pd.read_csv(f'{path}/{source}.csv', dtype={'building_id':np.uint16, 'meter':np.uint8}, parse_dates=['timestamp'])
    df = df.merge(building, on='building_id', how='left')
    df = df.merge(weather, on=['site_id', 'timestamp'], how='left')
    return df



### Training data:

In [ ]:
# load and display some samples
train = load_data('train')
train.sample(7)

### Test data:

In [ ]:
test = load_data('test')
test.sample(7)

### Data preprocessing:

Important: note that the target variable is computed as the logarithm of the column "meter_reading".

In [ ]:
# the counts above expose the missing data (Should we drop or refill the missing data?)
print("Ratio of available data (not NAN's):")
data_ratios = train.count()/len(train)
data_ratios

In [ ]:
class ASHRAE3Preprocessor(object):
    @classmethod
    def fit(cls, df, data_ratios=data_ratios):
        cls.avgs = df.loc[:,data_ratios < 1.0].mean()
        cls.pu_le = LabelEncoder()
        cls.pu_le.fit(df["primary_use"])

    @classmethod
    def transform(cls, df):
        df = df.fillna(cls.avgs) # refill NAN with averages
        df['primary_use'] = np.uint8(cls.pu_le.transform(df['primary_use']))  # encode labels

        # expand datetime into its components
        df['hour'] = np.uint8(df['timestamp'].dt.hour)
        df['day'] = np.uint8(df['timestamp'].dt.day)
        df['weekday'] = np.uint8(df['timestamp'].dt.weekday)
        df['month'] = np.uint8(df['timestamp'].dt.month)
        df['year'] = np.uint8(df['timestamp'].dt.year-2000)
        
        # parse and cast columns to a smaller type
        df.rename(columns={"square_feet": "log_square_feet"}, inplace=True)
        df['log_square_feet'] = np.float16(np.log(df['log_square_feet']))
        df['year_built'] = np.uint8(df['year_built']-1900)
        df['floor_count'] = np.uint8(df['floor_count'])
        
        # remove redundant columns
        for col in df.columns:
            if col in ['timestamp', 'row_id']:
                del df[col]
    
        # extract target column
        if 'meter_reading' in df.columns:
            df['meter_reading'] = np.log1p(df['meter_reading']).astype(np.float32) # comp metric uses log errors

        return df
        
ASHRAE3Preprocessor.fit(train)

In [ ]:
train = ASHRAE3Preprocessor.transform(train)
train.sample(7)

In [ ]:
train.dtypes

### Feature ranked correlation:

In [ ]:
%%time
fig, ax = plt.subplots(figsize=(16,8))
# use a ranked correlation to catch nonlinearities
corr = train[[col for col in train.columns if col != 'year']].sample(100100).corr(method='spearman')
_ = sns.heatmap(corr, annot=True,
                xticklabels=corr.columns.values,
                yticklabels=corr.columns.values)

In [ ]:
# force the model to use the weather data instead of dates, to avoid overfitting to the past history
features = [col for col in train.columns if col not in [target, 'year', 'month', 'day']]

### Train-validation partition:

In [ ]:
# Shuffle:
n = train.shape[0]
ix = np.random.permutation(n)

In [ ]:
# Training data:
ntrain = 15000000
nvalid = 5000000
tr_x, tr_y = train[features].iloc[ix[:ntrain]], train[target][ix[:ntrain]]
va_x, va_y = train[features].iloc[ix[ntrain:(ntrain+nvalid)]], train[target][ix[ntrain:(ntrain+nvalid)]]

In [ ]:
xtr = tr_x.values
ytr = tr_y.values
xval = va_x.values
yval = va_y.values

In [ ]:
print(xtr.shape)
print(ytr.shape)
print(xval.shape)
print(yval.shape)

### Variable normalization:

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
xtr = scaler.fit_transform(xtr)
xval = scaler.transform(xval)

In [ ]:
print(xtr.mean())
print(xtr.std())
print(xtr.shape)

print(xval.mean())
print(xval.std())
print(xval.shape)

### Neural network model with keras:

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras import backend as K

The next function computes the RMSE to evaluate the model. Note that, as we have taken the logarithm of the target, this value is equivalent to the RLMSE that kaggle uses for the evaluation. The function is from this notebook: https://www.kaggle.com/isaienkov/keras-nn-with-embeddings-for-cat-features-1-24 

In [ ]:
def rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true), axis=0))

In [ ]:
model = keras.Sequential()

# Linear model that obtains 1.90 in the public ranking:
#model.add(keras.layers.Dense(1, activation="linear", input_shape=(13,)))

# More complex network that obtains 1.27 in the public ranking:
model.add(keras.layers.Dense(1000, activation="relu", input_shape=(13,)))
#model.add(keras.layers.Dense(1000, activation="relu"))
model.add(keras.layers.Dense(100, activation="relu"))
model.add(keras.layers.Dense(1))

model.compile(optimizer=keras.optimizers.Adam(1.e-2), loss='mse', metrics=[rmse])
model.summary()

In [ ]:
nepochs = 6
history = model.fit(xtr, 
                    ytr, 
                    epochs=nepochs, 
                    validation_data=(xval, yval),
                    batch_size=10000)

In [ ]:
hd = history.history

epochs = range(1, nepochs+1)

plt.figure(figsize=(12,6))

plt.subplot(1,2,1)
plt.plot(epochs, hd['rmse'], "r", label="train")
plt.plot(epochs, hd['val_rmse'], "b", label="valid")
plt.grid(True)
plt.xlabel("epoch")
plt.ylabel("rmse")
plt.title("RMSE")
plt.legend()

plt.subplot(1,2,2)
plt.plot(epochs, hd['loss'], "r", label="train")
plt.plot(epochs, hd['val_loss'], "b", label="valid")
plt.grid(True)
plt.xlabel("epoch")
plt.ylabel("loss")
plt.title("Loss")
plt.legend()

plt.show()

### Predictions on the test set:

In [ ]:
test = ASHRAE3Preprocessor.transform(test)
test.sample(7)

In [ ]:
tst_x = test[features].iloc[:]

In [ ]:
xtst = tst_x.values
print(xtst.shape)

In [ ]:
xtst = scaler.transform(xtst)

In [ ]:
print(xtst.mean())
print(xtst.std())
print(xtst.shape)

In [ ]:
tst_preds = model.predict(xtst)
print(tst_preds.shape)

Note that we have to compute the exponential of the predictions, as we used the logarithm of the target to train the network:

In [ ]:
submission = pd.read_csv(f'{path}/sample_submission.csv')
submission['meter_reading'] = np.clip(np.expm1(tst_preds), a_min=0, a_max=None) # clip min at zero

In [ ]:
submission.head(9)

In [ ]:
submission.to_csv('submission.csv', index=False)